In [16]:
from LDAQ.micro_epsilon.acquisition import Scanner
# from acquisition import Scanner

import numpy as np
import matplotlib.pyplot as plt

Trenutno se za ta test ne rabi LDAQ (bi moral dejansko instalirati pyllt kot pise v navodilih v classu).

Tako da se lahko samo premakne tale notebook v `micro_epsilon` folder in tam skopira pyllt knjiznico (je na voljo v pyllt/pyllt).

V kolikor Scanner class dela, se ga lahko dokaj enostavno wrappa v MELaserScanner class. Ta class je trenutno zastarel in ga nisem posodabljal... Pojavi se namreč več težav:

- LDAQ bi postal dokaj počasen, ker bi trikrat izvajal kopiranje podatkov. Trenutno se izvaja dvakrat -> prvič na `ProfileBuffer`, podatki za trenutni profil, in nato še v `Buffer`, kjer so shranjeni vsi zajeti profili.
- v LDAQ je predvideno, da se pri branju vse skupaj zapakira v en array, tukaj pa pride do rahle težave, ker je za posamezen tip podatka predviden drug data type (X -> float, I -> int)

Mogoče bi lahko samo v read_loopu samo dodal dummy vrednosti, zato da je in potem povozil get_meas_dict metodo. Ne vem kako sprejemljivo je to...

Če je kaj prav bi se moral skener na tej točki sam povezati in biti pripravljen na zajem.

In [ ]:
device = Scanner()

Za začetek zajema je predvidena `start_measurement` metoda. Ta zažene nov thread, kjer se izvaja funkcija, ki zajema in shranjuje podatke. V kolikor se funkcija ne zažene v lastnem threadu pride do crasha. Ne znam razložiti zakaj, vem samo da mora biti generator za zajem instanciran znotraj iste funkcije, kjer se ga nato iterira.

Ta zahteva s seboj potegne precej težav... zato je za shranjevanje podatkov dodan nov class `Buffer`. V njemu so shranjeni vsi podatki s skenerja. Sam uporabnik načeloma nima nobene interakcije z njim. Za pridobitev podatkov je na voljo metoda `Scanner.get_data`.



In [ ]:
device.start_measurement()

start metoda praviloma nebi smela blokirati notebook-a. Zato se lahko uporabi `Scanner.stop_measurement` metoda za ustavitev zajema, ki nato tudi združi ustvarjen thread.


In [ ]:
device.stop_measurement()

Kot omenjeno je se podatke pridobi z naslednjo metodo. Ta vrne ven slovar s kanali in array-i z vrednostmi. Array-i so strukturirani kot [resolucija, i-ti profil].

In [ ]:
data = device.get_data()

Bi še omenil, da vrednosti za I, T in W so zelo verjetno napačne (vsaj na RPI je bil to problem). Pride do napake pri kalkulaciji vrednosti (napaka znotraj pyllt ali pa jaz kaj narobe delam). Če se iterira čez C array pa prideš do pravih vrednosti...

In [ ]:
plt.plot(data["X"], data["Z"])

# zadnja dva piksla nista definirana, napaka v firmwaru? treba vprašati dobavitelja
plt.xlim(-50, 50)
plt.ylim(70, 125)